# Primer modelo "serio", aprendiendo Fashion-MNIST

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from torchviz import make_dot
import torch.optim as optim

### Ejemplo 4: 

Podemos pensar una red neuronal como un *módulo* compuesto de otros *módulos* (p. ej. las capas).
El namespace (espacio de nombres) `torch.nn` provee de diversos building blocks (bloques de construcción) que facilitan la creación de redes neuronales de arquitecturas complejas por medio de la composición de módulos.
Más precisamente, PyTorch provee la clase `nn.Module`, la cual podemos subclasear para definir módulos propios.

Veamos un ejemplo de uso, en donde construiremos una red neuronal para clasificar imágenes del dataset FashionMNIST.

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Primero chequeamos si hay disponibles placas GPU con CUDA, o no.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Usando el dispositivo {}'.format(device))

Luego, definimos nuestro modelo como una clase derivada de la clase `nn.Module`.
Inicializamos la red neuronal usando el metodo`.__init__()`.
Toda subclase de `nn.Module` implementa el como operar sobre datos de entrada (input) para generar una salida (output) a travéz del método `.forward()`.
Por otro lado, notar que NO implementamos el método `.backward()`.

---

**Intermezzo.** Clases base y derivada. Las clases son el concepto central de la programación orientada a objetos.
Las clases definen la interface entre  los objetos de una dada *clase* y el resto del programa.
Esta inteface está compuesta por miembros.
Hay, básicamente, dos tipos de miembros: los métodos (funciones) y los atributos.

In [ ]:
# Definamos una clase
class A:
    def __init__(self,nombre):
        self.nombre = nombre
    def __str__(self):
        return f"Nombre={self.nombre}"
    def uppercase(self):
        return self.nombre.upper()
    def say_hi(self):
        print("Hi!")

# E instanciemos un objeto de la misma
a = A("mi-objeto")

In [ ]:
# Inspeccionemos su atributo .nombre
a.nombre

In [ ]:
# Invoquemos su método .uppercase()
a.uppercase()

In [ ]:
# Invoquemos su método .say_hi()
a.say_hi()

In [ ]:
# Invoquemos el método .__str__()
print(a)

In [ ]:
# Definamos una clase derivada
class Z(A):
    def __init__(self,nombre,cantidad):
        super(Z,self).__init__(nombre) # Llamemos al inicializador de la clase base (o super-clase).
        self.cantidad=cantidad
    def __str__(self):
        return f"Nombre={self.nombre}, Cantidad={self.cantidad}"
    def add_n(self,n):
        return self.cantidad+n
    def say_hi(self): # Redefinamos el método .say_hi()
        print("Hello!")
    
# Instanciemos un objeto de la clase Z
z = Z("mi-otro-objeto",10)
# e invoquemos el método .__str__()
print(z)

In [ ]:
# Veamos el atributo .cantidad
z.cantidad

In [ ]:
# Y también el miembro heredado .nombre
z.nombre

In [ ]:
# Invoquemos el método .add1()
z.add_n(5)

In [ ]:
# Invoquemos, también, el método .uppercase()

In [ ]:
z.uppercase()

In [ ]:
# Y, finalmente, invoquemos el método heredado y redefinido .say_hi()
z.say_hi()

Vemos que los métodos (y atributos) heredados se comportantal como fueron definidos en la clase base (`A` en este caso), salvo que se los redefinan en la clase derivada (`Z` en este caso).

Las clases bases son también conocidas como super-clases y las clases derivadas como sub-clases.
Una clase puede tener varias super-clases.
De una clase pueden derivarse varias sub-clases.

Si una clase A tiene una super clase B, y B tiene una superclase C, entonces C es una superclase (no inmediata) de A.
Decimos que existe una dependencia circular entre dos clases A y B, si A es una superclase de B y B una superclase de A.
Hay que evitar dependencias circulares entre clases.

---

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Primero convertimos los datos de entrada a un gran vector.
        self.flatten = nn.Flatten()
        # Luego generamos una secuencia de transformaciones que implementan las capas de la red.
        # Algunas capas consisten en transformaciones lineales, que incluyen parámetros.
        # Otras consisten en funciones de activación no lineales (en este caso de tipo ReLU), que no incluyen parámetros.
        # Los tamaños de las capas se especifican a travez de los tamaños de las transformaciones lineales.
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        # Este método es usado internamente para transformar inputs en outputs en las fases forward.
        # Al igual que antes, aplanamos los datos de entrada en un gran vector.
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

---

**Intermezzo.** Recordemos que es una *logit*. En estadística, la *logit* es la inversa de la función *logística*

$$ \sigma(x) = \frac{1}{1+e^{-x}} $$

donde $x\in (-\infty,\infty)$ y $\sigma(x)\in (0,1)$. En particular, $\sigma(x\to -\infty) = 0$ y $\sigma(x\to \infty) = 1$.
Por ende, 

$$ \mathsf{logit}(p) = \sigma^{-1}(p) = \ln\bigg(\frac{p}{1-p}\bigg) $$

donde $p\in (0,1)$ y $\mathsf{logit}(p)\in (-\infty,\infty)$. En particular, $\mathsf{logit}(p\to 0^+) \to -\infty$ y $\mathsf{logit}(p\to 1^-) \to \infty$.

La *logística* es una función creciente que convierte un número real en un número entre 0 y 1.

---

Luego creamos una un objeto (o instancia) de la clase `NeuralNetwork`, y lo movemos al dispositivo que usaremos.
Además, imprimimos su estructura para ver inspeccionar.

In [ ]:
model = NeuralNetwork().to(device)
print(model)

---

**Intermezzo** Recordemos que la función $\mathsf{softmax} \in \mathbb{R}^n\to \mathbb{R}^n$ viene dada por

$$ \mathsf{softmax}_i(v) = \frac{e^{v_i}}{\sum_j e^{v_j}} $$

donde $v=(v_1,v_2,...,v_n)$ y $v_i \in (-\infty,\infty)$. 
Notar, $\sum_i \mathsf{softmax}_i(v) = 1$.
La función $\mathsf{softmax}$ provee una forma de convertir una tupla de números reales a una distribución de probabilidades acorde.

---

Para usar el modelo, le pasamos datos de entrada.
Esto ejecuta el método `.forward()`, junto con algunas otras operaciones de fondo.
Ojo! No hay que llamar `model.forward()` directamente!

Tras llamar el modelo sobre algunos inputs, este retorna un tensor 10-dimensional con algunos valores crudos o *raw* predecidos para cada clase.
Obtenemos de estos probabilidades de predicción por medio de pasarlos a travéz de una instancia del modulo `nn.Softmax`.

Veamos un ejemplo

In [ ]:
# Generamos una muestra de entrada trucha que imite a FashionMNIST (una imagen de píxeles aleatorios).
X = torch.rand(1, 28, 28, device=device)

# Se los damos de comer al modelo via el método .__call__() del mismo, que hereda de la clase nn.Module.
# Eventualmente, el método .__call__() se encargará de llamar al método .forward() que nosotros hemos 
# implementado.
# Notar que este modelo no está entrenado, por lo que el output será "cualquier cosa".
logits = model(X)

# Imprimimos la salida cruda (raw) generada por el modelo
print(f"Salida del modelo = {logits}")
print("")

# Usando la función "softmax", convertimos estos valores crudos a probabilidades y las imprimimos
pred_probab = nn.Softmax(dim=1)(logits)
print(f"Probabilidades predecidas = {pred_probab}")
print("")

# Elegimos la categoría asociada a la mayor probabilidad.
y_pred = pred_probab.argmax(1)
print(f"Categoría predecida: {y_pred}")

Ahora, veamos como entrenar el modelo con el dataset *FashionMNIST*.

https://github.com/zalandoresearch/fashion-mnist

Este dataset se implementa en términos de las clases `Dataset` y `DataLoader` proveidas por PyTorch.
Respecitivamente, estas clases se usan para cargar y muestrear conjuntos de datos de formas convenientes y compatibles con PyTorch.

Comenzamos por cargar el dataset de entrenamiento (train) y el dataset de prueba (test).

In [ ]:
# La primera vez esto tarda un rato ya que tiene que bajar los datos de la red.

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Para entender mejor que son estos objetos, veamos de que clases derivan

In [ ]:
training_data.__class__.__mro__

In [ ]:
test_data.__class__.__mro__

Para entender como funciona la clase `Dataset`, veamos brevemente un ejemplo de una clase personalizada que se deriva de la misma.

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

# Por el momento, a esta clase la definimos pero no la usamos.
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    # Redefinimos el método .__len__()
    def __len__(self):
        return len(self.img_labels)

    # Redefinimos el método .__getitem__()    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

Exploremos un poco los datos de FashionMNIST.

In [ ]:
# Las muestras de FashionMNIST consisten en imágenes en 256 escalas de grises de 28x28 píxeles cada una.
# Las imágenes constituyen prendas, calzados, etc.
# Cada imagen tiene asociado un etiqueta (label) que puede adoptar un valor de entre 10 opciones.
# Cada etiqueta representa una de las 10 categorías en las que las imágenes fueron clasificadas.
# El objetivo es aprender a clasificar éstas imágenes de acuerdo a las etiquetas asociadas.

# Las etiquetas son
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

# Cada dataset puede ser indexado.
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

Generemos ahora instancias de la clase `DataLoader` para estos objetos derivados de la clase `DataSet`.

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=9)
test_dataloader = DataLoader(test_data, batch_size=9)

Veamos como estos objetos funcionan

In [ ]:
# Generamos un iterador a partir del objeto "train_dataloader"

train_dataloader_iterable = iter(train_dataloader)

In [ ]:
# Pedimos un batch

train_features, train_labels = next(train_dataloader_iterable)
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

# Graficamos las imágenes del batch.

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(cols * rows):
    img   = train_features[i]
    label = train_labels[i]
    figure.add_subplot(rows, cols, i+1)
    plt.title(labels_map[label.item()])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
    #print(f"Label: {label}")
plt.show()

Ya estamos en condiciones de implementar el entrenador del modelo.

In [ ]:
# Definimos la función de entrenamiento
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    # Iteramos sobre lotes (batchs)
    for batch, (X, y) in enumerate(dataloader):
        # Calculamos la predicción del modelo y la correspondiente pérdida (error)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagamos usando el optimizador proveido.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Imprimimos el progreso...
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"batch={batch} loss={loss:>7f}  muestras-procesadas:[{current:>5d}/{size:>5d}]")

# De manera similar, definimos la función de testeo
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Para testear, desactivamos el cálculo de gradientes.
    with torch.no_grad():
        # Iteramos sobre lotes (batches)
        for X, y in dataloader:
            # Calculamos las predicciones del modelo...
            pred = model(X)
            # y las correspondientes pérdidas (errores), los cuales vamos acumulando en un valor total.
            test_loss += loss_fn(pred, y).item()
            # También calculamos el número de predicciones correctas, y lo acumulamos en un total.
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Calculamos la pérdida total y la fracción de clasificaciones correctas, y las imprimimos.
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# Definimos los hiperparámetros del entrenamiento

learning_rate = 1e-3
batch_size = 64
epochs = 5

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
# Creamos una instancia de una función de pérdida, una entropy loss en este caso
loss_fn = nn.CrossEntropyLoss()
# y un optimizador, un Stochastic Gradient Descent, en este caso.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Finalmente, entrenamos iterando sobre épocas
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")